In [2]:
from PIL import Image, ImageDraw
import math
import os

# --- Configuration ---
GIF_FILENAME = "vector_construction.gif"
OUTPUT_FOLDER = "assets"
IMAGE_SIZE = (600, 600)  # Increased size for better clarity
BACKGROUND_COLOR = (255, 255, 255)  # White
AXES_COLOR = (0, 0, 0)  # Black
GUIDE_LINE_COLOR = (200, 200, 200) # Light gray for construction lines
VECTOR_COLOR = (0, 0, 255)  # Blue
CURRENT_VECTOR_COLOR = (0, 100, 255) # Slightly lighter blue for current segment
CENTER = (IMAGE_SIZE[0] // 2, IMAGE_SIZE[1] // 2)
SCALE = 50  # Pixels per unit in 3D space
NUM_SUB_FRAMES_PER_STEP = 20 # How many frames for each animation segment (X, Y, Z)
TOTAL_STEPS = 3 # X, Y, Z
DURATION_PER_FRAME_MS = 60 # Milliseconds per frame (faster transition)

# The target vector components
TARGET_VEC = (2, 1, 3) # (x, y, z)

# --- Create output folder if it doesn't exist ---
os.makedirs(OUTPUT_FOLDER, exist_ok=True)
full_gif_path = os.path.join(OUTPUT_FOLDER, GIF_FILENAME)

# --- Fixed View Rotation (Static Camera) ---
# These values define the fixed perspective of our 3D space
# Rotate around X-axis for tilt, Y-axis for left-right perspective
FIXED_ROTATION_X = math.radians(30) # Tilt up
FIXED_ROTATION_Y = math.radians(-45) # Rotate left
FIXED_ROTATION_Z = 0 # No Z-axis rotation

# --- 3D to 2D Projection (with Fixed Camera) ---
def project_3d_to_2d(x, y, z):
    # Apply fixed rotations
    # Rotate around X-axis
    y_prime = y * math.cos(FIXED_ROTATION_X) - z * math.sin(FIXED_ROTATION_X)
    z_prime = y * math.sin(FIXED_ROTATION_X) + z * math.cos(FIXED_ROTATION_X)
    y, z = y_prime, z_prime

    # Rotate around Y-axis
    x_prime = x * math.cos(FIXED_ROTATION_Y) + z * math.sin(FIXED_ROTATION_Y)
    z_prime = -x * math.sin(FIXED_ROTATION_Y) + z * math.cos(FIXED_ROTATION_Y)
    x, z = x_prime, z_prime
    
    # Scale and translate to image coordinates
    return int(CENTER[0] + x * SCALE), int(CENTER[1] - y * SCALE) # -y for Y-axis up

# --- Draw Functions ---
def draw_arrowhead(draw, p1, p2, color, arrow_size=10, width=4):
    """Draws an arrowhead at p2 pointing from p1."""
    if p1 == p2: # Avoid division by zero for zero-length vectors
        return

    angle = math.atan2(p2[1] - p1[1], p2[0] - p1[0])
    
    # Points for the arrowhead
    arrow_p1 = (p2[0] - arrow_size * math.cos(angle - math.pi / 6),
                p2[1] - arrow_size * math.sin(angle - math.pi / 6))
    arrow_p2 = (p2[0] - arrow_size * math.cos(angle + math.pi / 6),
                p2[1] - arrow_size * math.sin(angle + math.pi / 6))
    
    # Draw the main line
    draw.line([p1, p2], fill=color, width=width)
    
    # Draw the arrowhead lines
    draw.line([p2, arrow_p1], fill=color, width=width)
    draw.line([p2, arrow_p2], fill=color, width=width)


def draw_static_axes(draw):
    # Draw X-axis
    origin_2d = project_3d_to_2d(0, 0, 0)
    x_end_2d = project_3d_to_2d(3, 0, 0)
    draw_arrowhead(draw, origin_2d, x_end_2d, AXES_COLOR, arrow_size=8, width=2)
    
    # Draw Y-axis
    y_end_2d = project_3d_to_2d(0, 3, 0)
    draw_arrowhead(draw, origin_2d, y_end_2d, AXES_COLOR, arrow_size=8, width=2)
    
    # Draw Z-axis
    z_end_2d = project_3d_to_2d(0, 0, 3)
    draw_arrowhead(draw, origin_2d, z_end_2d, AXES_COLOR, arrow_size=8, width=2)

# --- Generate Frames ---
frames = []

# --- Stage 0: Show only axes for a few frames ---
for _ in range(5):
    img = Image.new('RGB', IMAGE_SIZE, BACKGROUND_COLOR)
    draw = ImageDraw.Draw(img)
    draw_static_axes(draw)
    frames.append(img)

# --- Stage 1: Move along X-axis ---
for i in range(NUM_SUB_FRAMES_PER_STEP + 1):
    img = Image.new('RGB', IMAGE_SIZE, BACKGROUND_COLOR)
    draw = ImageDraw.Draw(img)
    draw_static_axes(draw)

    # Current X component
    current_x = TARGET_VEC[0] * (i / NUM_SUB_FRAMES_PER_STEP)
    
    # Path taken (from origin to current X)
    p0 = project_3d_to_2d(0, 0, 0)
    px = project_3d_to_2d(current_x, 0, 0)
    draw_arrowhead(draw, p0, px, CURRENT_VECTOR_COLOR, width=4)

    # Draw the "final" vector up to this point
    draw_arrowhead(draw, p0, px, VECTOR_COLOR, width=5)
    
    frames.append(img)

# --- Stage 2: Move along Y-axis ---
for i in range(NUM_SUB_FRAMES_PER_STEP + 1):
    img = Image.new('RGB', IMAGE_SIZE, BACKGROUND_COLOR)
    draw = ImageDraw.Draw(img)
    draw_static_axes(draw)

    # X-component is now full
    full_x_point = (TARGET_VEC[0], 0, 0)
    
    # Current Y component
    current_y = TARGET_VEC[1] * (i / NUM_SUB_FRAMES_PER_STEP)
    
    # Draw the first segment (along X)
    p0 = project_3d_to_2d(0, 0, 0)
    px_full = project_3d_to_2d(*full_x_point)
    draw_arrowhead(draw, p0, px_full, VECTOR_COLOR, width=5)
    
    # Draw the current Y segment
    py = project_3d_to_2d(full_x_point[0], current_y, 0)
    draw_arrowhead(draw, px_full, py, CURRENT_VECTOR_COLOR, width=4)
    
    # Draw the combined "final" vector up to this point
    draw_arrowhead(draw, p0, py, VECTOR_COLOR, width=5)

    frames.append(img)

# --- Stage 3: Move along Z-axis ---
for i in range(NUM_SUB_FRAMES_PER_STEP + 1):
    img = Image.new('RGB', IMAGE_SIZE, BACKGROUND_COLOR)
    draw = ImageDraw.Draw(img)
    draw_static_axes(draw)

    # X and Y components are now full
    full_x_y_point = (TARGET_VEC[0], TARGET_VEC[1], 0)
    
    # Current Z component
    current_z = TARGET_VEC[2] * (i / NUM_SUB_FRAMES_PER_STEP)
    
    # Draw the first segment (along X)
    p0 = project_3d_to_2d(0, 0, 0)
    px_full = project_3d_to_2d(TARGET_VEC[0], 0, 0)
    draw_arrowhead(draw, p0, px_full, VECTOR_COLOR, width=5)
    
    # Draw the second segment (along Y)
    p_x_end = project_3d_to_2d(TARGET_VEC[0], 0, 0) # Point after X
    p_xy_end = project_3d_to_2d(TARGET_VEC[0], TARGET_VEC[1], 0) # Point after Y
    draw_arrowhead(draw, p_x_end, p_xy_end, VECTOR_COLOR, width=5)

    # Draw the current Z segment
    pz = project_3d_to_2d(full_x_y_point[0], full_x_y_point[1], current_z)
    draw_arrowhead(draw, p_xy_end, pz, CURRENT_VECTOR_COLOR, width=4)
    
    # Draw the full "final" vector up to this point
    draw_arrowhead(draw, p0, pz, VECTOR_COLOR, width=5)

    frames.append(img)

# --- Hold the final image for a few frames ---
for _ in range(10):
    frames.append(frames[-1])


# --- Save as GIF ---
if frames:
    frames[0].save(
        full_gif_path,
        format="GIF",
        append_images=frames[1:],
        save_all=True,
        duration=DURATION_PER_FRAME_MS,
        loop=0 # Loop forever
    )
    print(f"GIF saved successfully to: {full_gif_path}")
else:
    print("No frames were generated.")

# Display the GIF (this will show the last frame as a static image in Jupyter)
# For animation, you'd typically open the file from your system or embed it via HTML
# from IPython.display import Image as DisplayImage
# DisplayImage(filename=full_gif_path)

GIF saved successfully to: assets/vector_construction.gif


In [ ]:
from PIL import Image, ImageDraw, ImageFont
import math
import imageio.v2 as imageio # Using v2 for newer imageio versions

def create_speed_acceleration_gif(filename="speed_acceleration.gif", duration_per_frame=0.1):
    """
    Creates a GIF visualizing the concepts of speed and acceleration.
    """
    width, height = 800, 600
    background_color = (250, 250, 250) # Light gray
    object_color = (50, 50, 50) # Dark gray
    velocity_color = (60, 150, 255) # Blue
    acceleration_color = (255, 90, 90) # Red
    text_color = (30, 30, 30)

    object_radius = 10
    base_arrow_length = 30
    arrow_head_size = 8
    
    # Load a font
    try:
        font_label = ImageFont.truetype("arial.ttf", 14)
        font_scenario = ImageFont.truetype("arial.ttf", 18)
    except IOError:
        print("Warning: Arial font not found. Using default Pillow font.")
        font_label = ImageFont.load_default()
        font_scenario = ImageFont.load_default()

    frames = []
    num_frames = 60 # Total frames for the entire GIF

    def draw_arrow(draw_obj, start_pos, end_pos, color):
        """Helper to draw a line with an arrowhead."""
        draw_obj.line(start_pos + end_pos, fill=color, width=2)
        
        # Calculate arrow head points
        angle = math.atan2(end_pos[1] - start_pos[1], end_pos[0] - start_pos[0])
        p1 = (end_pos[0] - arrow_head_size * math.cos(angle - math.pi/6),
              end_pos[1] - arrow_head_size * math.sin(angle - math.pi/6))
        p2 = (end_pos[0] - arrow_head_size * math.cos(angle + math.pi/6),
              end_pos[1] - arrow_head_size * math.sin(angle + math.pi/6))
        draw_obj.polygon([end_pos, p1, p2], fill=color)

    for frame_idx in range(num_frames):
        current_image = Image.new('RGB', (width, height), background_color)
        draw = ImageDraw.Draw(current_image)

        # --- Top-level Title ---
        draw.text((width // 2, 20), "Speed vs. Acceleration", fill=text_color, font=ImageFont.truetype("arial.ttf", 24) if "arial.ttf" in str(font_label.path) else font_label, anchor="mm")

        # --- Scenario 1: Constant Velocity (Constant Speed, Zero Acceleration) ---
        scenario1_y = 120
        draw.text((50, scenario1_y - 30), "1. Constant Velocity (Constant Speed)", fill=text_color, font=font_scenario)
        
        t1 = (frame_idx % (num_frames // 3)) / (num_frames // 3 - 1) # Time for scenario 1
        pos1_x = 100 + t1 * 200
        pos1_y = scenario1_y + 50
        
        draw.ellipse((pos1_x - object_radius, pos1_y - object_radius, pos1_x + object_radius, pos1_y + object_radius), fill=object_color)
        draw_arrow(draw, (pos1_x, pos1_y), (pos1_x + base_arrow_length, pos1_y), velocity_color)
        draw.text((pos1_x + base_arrow_length + 5, pos1_y - 10), "Velocity", fill=velocity_color, font=font_label)
        draw.line((100, pos1_y, 300, pos1_y), fill=(200, 200, 200), width=1) # Path


        # --- Scenario 2: Increasing Speed (Constant Acceleration) ---
        scenario2_y = 300
        draw.text((50, scenario2_y - 30), "2. Increasing Speed (Acceleration)", fill=text_color, font=font_scenario)
        
        t2 = (frame_idx % (num_frames // 3)) / (num_frames // 3 - 1) # Time for scenario 2
        # Start slow, accelerate
        current_speed = 0.5 + t2 * 1.5 # Speed increases from 0.5 to 2
        pos2_x = 100 + (0.5 * t2 + 0.5 * t2**2) * 150 # Position based on acceleration
        pos2_y = scenario2_y + 50
        
        draw.ellipse((pos2_x - object_radius, pos2_y - object_radius, pos2_x + object_radius, pos2_y + object_radius), fill=object_color)
        
        # Velocity arrow grows with speed
        draw_arrow(draw, (pos2_x, pos2_y), (pos2_x + base_arrow_length * current_speed / 2, pos2_y), velocity_color)
        draw.text((pos2_x + base_arrow_length * current_speed / 2 + 5, pos2_y - 10), "Velocity", fill=velocity_color, font=font_label)
        
        # Constant acceleration arrow in same direction
        draw_arrow(draw, (pos2_x, pos2_y), (pos2_x + base_arrow_length * 0.5, pos2_y + 20), acceleration_color) # Slightly offset for clarity
        draw.text((pos2_x + base_arrow_length * 0.5 + 5, pos2_y + 10), "Acceleration", fill=acceleration_color, font=font_label)
        draw.line((100, pos2_y, 300, pos2_y), fill=(200, 200, 200), width=1) # Path


        # --- Scenario 3: Constant Speed, Changing Direction (Acceleration) ---
        scenario3_y = 480
        draw.text((50, scenario3_y - 30), "3. Turning (Constant Speed, Acceleration)", fill=text_color, font=font_scenario)
        
        t3 = (frame_idx % (num_frames // 3)) / (num_frames // 3) * (math.pi / 2) # Angle for quarter circle
        
        center_x = 200
        center_y = scenario3_y + 50
        radius = 80
        
        # Path (quarter circle)
        draw.arc((center_x - radius, center_y - radius, center_x + radius, center_y + radius),
                 start=90, end=0, fill=(200, 200, 200), width=1)
        
        pos3_x = center_x + radius * math.sin(t3)
        pos3_y = center_y - radius * math.cos(t3)
        
        draw.ellipse((pos3_x - object_radius, pos3_y - object_radius, pos3_x + object_radius, pos3_y + object_radius), fill=object_color)
        
        # Velocity arrow (tangential, constant length)
        vel_x = math.cos(t3) * base_arrow_length / 2
        vel_y = math.sin(t3) * base_arrow_length / 2
        draw_arrow(draw, (pos3_x, pos3_y), (pos3_x + vel_x, pos3_y + vel_y), velocity_color)
        draw.text((pos3_x + vel_x + 5, pos3_y - 10), "Velocity", fill=velocity_color, font=font_label)
        
        # Acceleration arrow (towards center of circle)
        acc_x = -math.sin(t3) * base_arrow_length / 2 * 0.8
        acc_y = math.cos(t3) * base_arrow_length / 2 * 0.8
        draw_arrow(draw, (pos3_x, pos3_y), (pos3_x + acc_x, pos3_y + acc_y), acceleration_color)
        draw.text((pos3_x + acc_x + 5, pos3_y + 10), "Acceleration", fill=acceleration_color, font=font_label)

        frames.append(current_image)

    # Save as GIF
    imageio.mimsave(filename, frames, duration=duration_per_frame, loop=0) # loop=0 means loop indefinitely
    print(f"GIF visualization saved as {filename}")

# Create the GIF visualization
create_speed_acceleration_gif()